# This is a basic Starter Kernel for the New York City Taxi Fare Prediction Playground Competition 
Here we'll use a simple linear model based on the travel vector from the taxi's pickup location to dropoff location which predicts the `fare_amount` of each ride.

This kernel uses some `pandas` and mostly `numpy` for the critical work.  There are many higher-level libraries you could use instead, for example `sklearn` or `statsmodels`.  

In [ ]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('../input'))

### Setup training data
First let's read in our training data.  Kernels do not yet support enough memory to load the whole dataset at once, at least using `pd.read_csv`.  The entire dataset is about 55M rows, so we're skipping a good portion of the data, but it's certainly possible to build a model using all the data.

In [ ]:
train_df =  pd.read_csv('../input/train.csv', nrows = 10_000_000)
train_df.dtypes

In [ ]:
test_df = pd.read_csv('../input/test.csv')
test_df.dtypes

In [ ]:
test_df.shape

Let's create two new features in our training set representing the "travel vector" between the start and end points of the taxi ride, in both longitude and latitude coordinates.  We'll take the absolute value since we're only interested in distance traveled. Use a helper function since we'll want to do the same thing for the test set later.

In [ ]:
# Given a dataframe, add two new features 'abs_diff_longitude' and
# 'abs_diff_latitude' reprensenting the "Manhattan vector" from
# the pickup location to the dropoff location.
def add_travel_vector_features(df):
    df['abs_diff_longitude'] = (df.dropoff_longitude - df.pickup_longitude).abs()
    df['abs_diff_latitude'] = (df.dropoff_latitude - df.pickup_latitude).abs()

add_travel_vector_features(train_df)
add_travel_vector_features(test_df)

### Explore and prune outliers
First let's see if there are any `NaN`s in the dataset.

In [ ]:
print(train_df.isnull().sum())

There are a small amount, so let's remove them from the dataset.

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df.dropna(how = 'any', axis = 'rows')
print('New size: %d' % len(train_df))

Now let's quickly plot a subset of our travel vector features to see its distribution.

In [ ]:
plot = train_df.iloc[:2000].plot.scatter('abs_diff_longitude', 'abs_diff_latitude')

We expect most of these values to be very small (likely between 0 and 1) since it should all be differences between GPS coordinates within one city.  For reference, one degree of latitude is about 69 miles.  However, we can see the dataset has extreme values which do not make sense.  Let's remove those values from our training set. Based on the scatterplot, it looks like we can safely exclude values above 5 (though remember the scatterplot is only showing the first 2000 rows...)

In [ ]:
print('Old size: %d' % len(train_df))
train_df = train_df[(train_df.abs_diff_longitude < 5.0) & (train_df.abs_diff_latitude < 5.0)]
print('New size: %d' % len(train_df))

**Now let's extract pickup time from date_time column of dataset**

In [ ]:
ls1 = list(train_df['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][11:-7:]
train_df['pickuptime'] = ls1

ls1 = list(test_df['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][11:-7:]
test_df['pickuptime'] = ls1

In [ ]:
train_df.head()

**Now let's slice date & time from pickup_datetime column and convert it into weekdays .Here 0 = monday and 6 = Sunday**** 

In [ ]:
ls1 = list(train_df['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][:-4:]
    ls1[i] = pd.Timestamp(ls1[i])
    ls1[i] = ls1[i].weekday()
train_df['weekday'] = ls1


ls1 = list(test_df['pickup_datetime'])
for i in range(len(ls1)):
    ls1[i] = ls1[i][:-4:]
    ls1[i] = pd.Timestamp(ls1[i])
    ls1[i] = ls1[i].weekday()
test_df['weekday'] = ls1

In [ ]:
train_df.head()

**Now we can drop the pickup_datetime column**

In [ ]:
train_df.drop('pickup_datetime', inplace = True, axis = 1)
test_df.drop('pickup_datetime', inplace = True, axis = 1)

**Now let's replace the weekday value of 0 , 1 , etc with monday , tuesday , etc**

In [ ]:
train_df['weekday'].replace(to_replace = [i for i in range(0,7)], value = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)
test_df['weekday'].replace(to_replace = [i for i in range(0,7)], value = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], inplace=True)

In [ ]:
train_df.head()

**Now let's apply One hot Encoding**

In [ ]:
train_one_hot = pd.get_dummies(train_df['weekday'])
test_one_hot = pd.get_dummies(test_df['weekday'])
train_df = pd.concat([train_df, train_one_hot], axis = 1)
test_df = pd.concat([test_df, test_one_hot], axis = 1)

**Now as we already use OneHotEncoding in weekday column its time to drop the weekday column**

In [ ]:
train_df.drop('weekday', inplace=True, axis=1)
test_df.drop('weekday', inplace=True, axis=1)

In [ ]:
train_df.head()

**Now let's convert the pickuptime into integer**

In [ ]:
ls1 = list(train_df['pickuptime'])
for i in range(len(ls1)):
    z = ls1[i].split(':')
    ls1[i] = int(z[0])*100+int(z[1])
train_df['pickuptime'] = ls1
    
    
ls1 = list(test_df['pickuptime'])
for i in range(len(ls1)):
    z = ls1[i].split(':')
    ls1[i] = int(z[0])*100+int(z[1])
test_df['pickuptime'] = ls1

In [ ]:
train_df.head()

**The below mention code is what I searched in google , actually this code will conervt our given pickup/dropoff longitude/latitude in Kilometer.**

In [ ]:
R = 6373.0
lat1 = np.asarray(np.radians(train_df['pickup_latitude']))
lon1 = np.asarray(np.radians(train_df['pickup_longitude']))
lat2 = np.asarray(np.radians(train_df['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_df['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1 = []

a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R * c

train_df['Distance'] = np.asarray(distance)*0.621




lat1 = np.asarray(np.radians(test_df['pickup_latitude']))
lon1 = np.asarray(np.radians(test_df['pickup_longitude']))
lat2 = np.asarray(np.radians(test_df['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_df['dropoff_longitude']))

dlon = lon2 - lon1
dlat = lat2 - lat1
ls1 = []

a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
distance = R * c

test_df['Distance'] = np.asarray(distance)*0.621

In [ ]:
train_df.head()

**As we all know that the airport cabs have higher price as compare to other cabs .So using same method as above I have added distance of pickup and drop from Airport by using airport longitude and latitude seperately for Airport of John F kennedy
Latitude: 40.6413111 , Longitude: -73.7781391**

In [ ]:
R = 6373.0
lat1 =np.asarray(np.radians(train_df['pickup_latitude']))
lon1 = np.asarray(np.radians(train_df['pickup_longitude']))
lat2 = np.asarray(np.radians(train_df['dropoff_latitude']))
lon2 = np.asarray(np.radians(train_df['dropoff_longitude']))

lat3=np.zeros(len(train_df))+np.radians(40.6413111)
lon3=np.zeros(len(train_df))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
train_df['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
train_df['Dropoff_Distance_airport']=np.asarray(distance2)*0.621



lat1 =np.asarray(np.radians(test_df['pickup_latitude']))
lon1 = np.asarray(np.radians(test_df['pickup_longitude']))
lat2 = np.asarray(np.radians(test_df['dropoff_latitude']))
lon2 = np.asarray(np.radians(test_df['dropoff_longitude']))

lat3=np.zeros(len(test_df))+np.radians(40.6413111)
lon3=np.zeros(len(test_df))+np.radians(-73.7781391)
dlon_pickup = lon3 - lon1
dlat_pickup = lat3 - lat1
d_lon_dropoff=lon3 -lon2
d_lat_dropoff=lat3-lat2
a1 = np.sin(dlat_pickup/2)**2 + np.cos(lat1) * np.cos(lat3) * np.sin(dlon_pickup/ 2)**2
c1 = 2 * np.arctan2(np.sqrt(a1), np.sqrt(1 - a1))
distance1 = R * c1
test_df['Pickup_Distance_airport']=np.asarray(distance1)*0.621

a2=np.sin(d_lat_dropoff/2)**2 + np.cos(lat2) * np.cos(lat3) * np.sin(d_lon_dropoff/ 2)**2
c2 = 2 * np.arctan2(np.sqrt(a2), np.sqrt(1 - a2))
distance2 = R * c2

    
test_df['Dropoff_Distance_airport']=np.asarray(distance2)*0.621


**Now let's roundup the above derive values upto 2 decimal places**

In [ ]:
train_df.head()

In [ ]:
train_df['Distance'] = np.round(train_df['Distance'],2)
train_df['Pickup_Distance_airport'] = np.round(train_df['Pickup_Distance_airport'],2)
train_df['Dropoff_Distance_airport'] = np.round(train_df['Dropoff_Distance_airport'],2)

test_df['Distance'] = np.round(test_df['Distance'],2)
test_df['Pickup_Distance_airport'] = np.round(test_df['Pickup_Distance_airport'],2)
test_df['Dropoff_Distance_airport'] = np.round(test_df['Dropoff_Distance_airport'],2)

In [ ]:
train_df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)
test_df.drop(['pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'],axis=1,inplace=True)

**NOTE**
* You may delete above mention column/ not , depends on the accuracy you are getting .
*  Now I am going to subtract the each value of longitude and lattittude with their respective mean
* Then , I will divide them with there variance

In [ ]:
train_df.head()

In [ ]:
train_df['abs_diff_longitude']=np.abs(train_df['abs_diff_longitude']-np.mean(train_df['abs_diff_longitude']))
train_df['abs_diff_longitude']=train_df['abs_diff_longitude']/np.var(train_df['abs_diff_longitude'])


train_df['abs_diff_latitude']=np.abs(train_df['abs_diff_latitude']-np.mean(train_df['abs_diff_latitude']))
train_df['abs_diff_latitude']=train_df['abs_diff_latitude']/np.var(train_df['abs_diff_latitude'])

In [ ]:
test_df.head()

In [ ]:
print(train_df.shape)
print(test_df.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X = train_df.drop(['key','fare_amount'],axis=1)
Y = train_df['fare_amount']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.01, random_state=80)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression(normalize = True)
lr.fit(X_train,Y_train)
print(lr.score(X_test,Y_test))

**As sample submission is of keys column first and then predict value with 2 decimal places**

In [ ]:
pred = np.round(lr.predict(test_df.drop('key', axis=1)), 2)

In [ ]:
submission = pd.DataFrame(data = pred , columns = ['fare_amount'])
submission['key'] = test_df['key']
submission = submission[['key', 'fare_amount']]          #change order of column header 

In [ ]:
submission.set_index('key', inplace=True)

In [ ]:
submission.to_csv('Submission.csv')